In [ ]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import pendulum
import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [3]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
p4p = P4P(market, market['lid'], market['lpwd'])

. storage markets.json
.\storage\markets.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!


In [4]:
def monitor(self, group='all', tid=None, socketio=None, tasks=None):
    if tid:
        tasks[tid]['is_running'] = True
        msg = {'name': 'P4P.monitor', 'tid': tid, 'group': group, 'is_last_run': tasks[tid]['is_last_run']}
        socketio.emit('event_task_start_running', msg, namespace='/markets', broadcast=True)

    try:
        print('Task Monitor start to run with group="' + group + '"')
        keywords = []
        # self.load_keywords('monitor')
        if tid:
            tasks[tid]['progress'] = 1
            socketio.emit('event_task_progress', {'tid': tid, 'progress': 1}, namespace='/markets', broadcast=True)
        with self.lock:
            self.load_url()
            if tid:
                tasks[tid]['progress'] = 2
                socketio.emit('event_task_progress', {'tid': tid, 'progress': 2}, namespace='/markets',
                              broadcast=True)
            # all_kws_count = int(self.browser.find_element_by_css_selector('a.all-kwcount span').text)
            print('switch to group:', group)
            all_kws_count = self.switch_to_group(group)
            #             time.sleep(5)
            kws_count = 0
            while True:
                css_selector = "div.keyword-manage .bp-table-main-wraper>table"
                table = self.browser.find_element_by_css_selector(css_selector)

                trs = table.find_elements_by_css_selector('tbody tr')

                has_checked = False
                for idx, tr in enumerate(trs):
                    kws_count += 1
                    if tid:
                        tasks[tid]['progress'] = int(kws_count / all_kws_count * 97) + 3
                        socketio.emit('event_task_progress', {'tid': tid, 'progress': tasks[tid]['progress']},
                                      namespace='/markets', broadcast=True)

                    print('index:', idx, len(trs), end=' > ')
                    ActionChains(self.browser).move_to_element(tr).perform()
                    id = tr.find_element_by_css_selector('td:first-child input').get_attribute('value').strip()
                    # if id not in self.keywords_list['monitor']:
                    #     print('skipped_not_in_monitoring')
                    #     continue

                    grp = tr.find_element_by_css_selector('td:nth-child(4)').text.strip()
                    if group != 'all':
                        if group not in grp:
                            print('skipped_not_in_group', group, grp)
                            continue

                    dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()

                    print(kws, end=' > ')

                    if not self.open_price_dialog(tr):
                        print('skipped_2')
                        continue

                    [prices, sponsors] = self.find_prices_and_sponsors(close=False)

                    if prices:
                        keywords.append([dt, id, kws, grp, prices, sponsors])

                    current_position = self.find_sponsor_list_position(sponsors=sponsors['sponsor_list'])
                    click_position = self.get_click_position(id, current_position)

                    if click_position != -1:
                        self.set_price(position=click_position)
                    else:
                        webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()

                    WebDriverWait(self.browser, 15).until(
                        EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.ui-mask')))

                    if not self.is_on(tr):
                        print('make selected', end=" > ")
                        tr.find_element_by_css_selector("td:first-child input").click()
                        has_checked = True

                    print(' >>>>>> successful end ')

                if has_checked:
                    css_selector = '.bp-table .toolbar a[data-role="btn-start"]'
                    btn_start = self.browser.find_element_by_css_selector(css_selector)
                    self.click(btn_start)
                    print('switch selected on.')

                    self.browser.implicitly_wait(1)
                    WebDriverWait(self.browser, 15).until(
                        EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.bp-loading-panel')))

                if not self.next_page():
                    break

        if keywords:
            self.save_crawling_result(keywords)
        if tid:
            tasks[tid]['progress'] = 100
            socketio.emit('event_task_progress', {'tid': tid, 'progress': 100}, namespace='/markets',
                          broadcast=True)
    except Exception as e:
        print('Error: ', e)
        traceback.print_exc()
    finally:
        if tid:
            tasks[tid]['is_running'] = False
            tasks[tid]['progress'] = 0
            if tasks[tid]['is_last_run']:
                del tasks[tid]
                socketio.emit('event_task_last_run_finished', {'tid': tid}, namespace='/markets', broadcast=True)

p4p.monitor = types.MethodType(monitor, p4p)

In [8]:
p4p.browser.set_window_size(1920, 1200)

In [ ]:
p4p.browser.save_screenshot('screenie.png')

In [9]:
group = '直通车App'

In [6]:
p4p.load_url()
p4p.switch_to_group(group)

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


NameError: name 'group' is not defined

In [12]:
p4p.crawl(group=group)

index: 0 8 > flare eyelash >  >>>>>> successful end 
index: 1 8 > flare eyelash extension >  >>>>>> successful end 
index: 2 8 > magnetic eyelashes >  >>>>>> successful end 
index: 3 8 > silk eyelashes >  >>>>>> successful end 
index: 4 8 > strip eyelash >  >>>>>> successful end 
index: 5 8 > synthetic eyelashes >  >>>>>> successful end 
index: 6 8 > volume eyelash >  >>>>>> successful end 
index: 7 8 > volume eyelash extension >  >>>>>> successful end 
next_page: False


In [10]:
p4p.monitor(group=group)

Task Monitor start to run with group="直通车App"
switch to group: 直通车App
index: 0 8 > flare eyelash >  >>>>>> successful end 
index: 1 8 > flare eyelash extension >  >>>>>> successful end 
index: 2 8 > magnetic eyelashes > make selected >  >>>>>> successful end 
index: 3 8 > silk eyelashes > make selected >  >>>>>> successful end 
index: 4 8 > strip eyelash > make selected >  >>>>>> successful end 
index: 5 8 > synthetic eyelashes > make selected >  >>>>>> successful end 
index: 6 8 > volume eyelash >  >>>>>> successful end 
index: 7 8 > volume eyelash extension >  >>>>>> successful end 
switch selected on.
next_page: False


In [11]:
p4p.turn_all_off(group=group)

next_page: False


In [ ]:
p4p.kws_tracking

In [ ]:
# p4p.load_url()

# css_selector = "div.keyword-manage .bp-table-main-wraper>table"
# table = p4p.browser.find_element_by_css_selector(css_selector)
# trs = table.find_elements_by_css_selector('tbody tr')
# idx = 1
# for tr in trs:
#     print()
#     print(idx, end=': ')
#     idx += 1
    
#     kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
#     print(kws, end=": ")
        
#     if not p4p.open_price_dialog(tr):
#         print('skipped_2')
#         idx += 1
#         continue
        
#     [prices, sponsors] = p4p.find_prices_and_sponsors()
#     print(prices, sponsors)
#     if idx > 10:
#         break

In [ ]:
tr = trs[8]
kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
print(kws, end=": ")

if not p4p.open_price_dialog(tr):
    print('skipped_2')
else:
#     time.sleep(1)
    prices = p4p.find_prices()

In [ ]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Tools']

In [ ]:
p4p.find_sponsor_list_position(kws='strip eyelash')

In [ ]:
group = '直通车App'

In [ ]:
p4p.monitor(group=group)

In [ ]:
p4p.kws_tracking = {}

In [ ]:
p4p.turn_all_off(group=group)

In [ ]:
text = "adsf"

In [ ]:
a,b = text.split(' ')